<a href="https://colab.research.google.com/github/smannan/LIFXBulbAnalysis/blob/main/PGEDataExtractFromInflux.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install influxdb-client

In [ ]:
import pandas as pd

In [12]:
from influxdb_client import InfluxDBClient, Point, WritePrecision

# You can generate a Token from the "Tokens Tab" in the UI
# https://us-central1-1.gcp.cloud2.influxdata.com/orgs/a33284f1a3112d0e/data-explorer
token = "uaElnni__FwThkojQgBlw069cZzqFiehCwog5QmiSsOKPSSdEj7O1PK8qDd_alm253SL8ZHj6PRhKuUclUXDDw=="
org = "praveenkumar23.anguru@gmail.com"
bucket = "PGE_CSV_DATA"
account_number = "863594456"

client = InfluxDBClient(url="https://us-central1-1.gcp.cloud2.influxdata.com", token=token, verify_ssl=False)

In [13]:
def query_field():
  # |> group(columns: ["_field"])
  # |> filter(fn:(r) => r._account_number == \"{account_number}\")
  # |> filter(fn:(r) => r._field =~ /cost|usage/)
  query = f"""
    from(bucket: \"{bucket}\")
    |> range(start: -365d)
    |> timeShift(duration: -7h)
    |> filter(fn:(r) => r._field =~ /cost|usage/)
    |> group(columns: ["_account_number"])
  """
  print (query)
  tables = client.query_api().query(query, org=org)
  return tables

In [14]:
def get_field_values_from_flux():
  tables = query_field()
  field_values = {}

  for table in tables:
    for record in table.records:
      if record.values['account_number'] == account_number:
        if record.values['_time'] not in field_values:
          field_values[record.values['_time']] = {
              'timestamp': record.values['_time'],
              'account_number': record.values['account_number']
          }
        
        field_values[record.values['_time']][record.values['_field']] = record.values['_value']

  return pd.DataFrame(field_values.values())

In [15]:
df = get_field_values_from_flux()


    from(bucket: "PGE_CSV_DATA")
    |> range(start: -365d)
    |> timeShift(duration: -7h)
    |> filter(fn:(r) => r._field =~ /cost|usage/)
    |> group(columns: ["_account_number"])
  


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [17]:
df.head()
print (len(df))

34091


In [18]:
df['timestamp'] = df['timestamp'].dt.round('15min')
df.head()

,timestamp,account_number,cost,usage
0,2020-10-10 20:30:00+00:00,863594456,0.02,0.10
1,2020-10-10 20:45:00+00:00,863594456,0.02,0.10
2,2020-10-10 21:00:00+00:00,863594456,0.02,0.10
3,2020-10-10 21:15:00+00:00,863594456,0.02,0.06
4,2020-10-10 21:30:00+00:00,863594456,0.02,0.06


In [19]:
min_time = str(df['timestamp'].min()).replace(" ", "-")
max_time = str(df['timestamp'].max()).replace(" ", "-")
df.to_csv('pge_data_from_influx_{0}_{1}.csv'.format(min_time, max_time))

In [20]:
min_time

'2020-10-10-20:30:00+00:00'

In [21]:
max_time

'2021-10-01-00:00:00+00:00'